In [6]:
import numpy as np 
from sklearn.model_selection import train_test_split
from DeepTCN.DeepTCN import DeepTCN
from DeepTCN.plots import plot_matplotlib
from data_process import get_data_loaders, getFeatures
from tensorflow.keras.models import load_model
# updated notebook

In [2]:
data = get_data_loaders()[0]
features = getFeatures()
y = data['energy_consumed'].values
x = data[features].values

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y = y.reshape(-1, 1)

[0.0041988  0.00420218 0.00420386 ... 0.41108662 0.41109065 0.41109251]


In [3]:
print("x shape: ", x.shape)
print("y shape: ", y.shape)

x shape:  (257896, 32)
y shape:  (257896, 1)


In [4]:
# Define the parameters for the DeepTCN model
forecast_period = 2  # You need to specify the forecast period you want to predict
lookback_period = 10  # You need to specify the number of past observations the model should look at
quantiles=[0.1, 0.5, 0.9]  # Specify the quantiles you are interested in predicting
filters = 32  # The number of filters in the TCN layers
kernel_size = 2  # The kernel size for the convolutional layers
dilation_rates = [1, 2, 4, 8]  # The dilation rates for the TCN layers
units = 64  # The number of units in the dense layer
loss = 'nonparametric'  # Specify the loss function to be used

# Initialize the DeepTCN model
model = DeepTCN(
    y=y,
    x=x,
    forecast_period=forecast_period,
    lookback_period=lookback_period,
    quantiles=quantiles,
    filters=filters,
    kernel_size=kernel_size,
    dilation_rates=dilation_rates,
    units=units,
    loss=loss
)

In [9]:
# Fit the model 
model.fit(
    learning_rate=0.001,
    batch_size=32,
    epochs=10,
    validation_split=0.2,
    verbose=1
)

model.save_model()

Epoch 1/10
6448/6448 [==============================] - 51s 8ms/step - loss: 0.0421 - val_loss: 0.0201
Epoch 2/10
6448/6448 [==============================] - 43s 7ms/step - loss: 0.0101 - val_loss: 0.0187
Epoch 3/10
6448/6448 [==============================] - 45s 7ms/step - loss: 0.0067 - val_loss: 0.0073
Epoch 4/10
6448/6448 [==============================] - 47s 7ms/step - loss: 0.0055 - val_loss: 0.0038
Epoch 5/10
6448/6448 [==============================] - 71s 11ms/step - loss: 0.0047 - val_loss: 0.0059
Epoch 6/10
6448/6448 [==============================] - 47s 7ms/step - loss: 0.0042 - val_loss: 0.0045
Epoch 7/10
6448/6448 [==============================] - 44s 7ms/step - loss: 0.0038 - val_loss: 0.0032
Epoch 8/10
6448/6448 [==============================] - 42s 7ms/step - loss: 0.0035 - val_loss: 0.0033
Epoch 9/10
6448/6448 [==============================] - 42s 7ms/step - loss: 0.0032 - val_loss: 0.0026
Epoch 10/10
6448/6448 [==============================] - 45s 7ms/step - 

In [44]:
"""
y: np.array.
            Past values of target time series, array with shape (n_samples, n_targets) where n_samples is the length
            of the time series and n_targets is the number of target time series. The number of past samples provided
            (n_samples) should not be less than the length of the lookback period.
            
x: np.array.
    Past and future values of features time series, array with shape (n_samples + n_forecast, n_features) where
    n_samples is the length of the time series, n_forecast is the decoder length and n_features is the number
    of features time series. The number of past samples provided (n_samples) should not be less than the length
    of the lookback period.
"""

# Predict the quantiles for the test set
# assume that the test set is the 50% to 60% quantile of the data set of flight 3
test_data = data[data["flight"] == 3]
total_samples = len(test_data)
start_index = int(total_samples * 0.5)
end_index = int(total_samples * 0.6)
n_samples = end_index - start_index + 1

# Select past data for y (lookback period)
y_past_end_index = start_index - 1
y_past_start_index = y_past_end_index - lookback_period + 1
y_past = test_data['energy_consumed'][y_past_start_index:y_past_end_index + 1].values.reshape(lookback_period, 1)

# Select past and future data for x (lookback period + forecast period)
x_end_index = end_index + forecast_period
x_start_index = y_past_start_index
x_test = test_data[features][x_start_index:x_end_index + 1].values.reshape(x_end_index - x_start_index + 1, len(features))

# Forecasting
#df = model.forecast(y=y_past, x=x_test)

print("y_past shape = ", y_past.shape)
print("x_test shape = ", x_test.shape)


y_past shape =  (10, 1)
x_test shape =  (133, 32)


In [46]:
df = model.forecast(y=y_test, x=x_test)
df

,time_idx,target_1,target_1_0.1,target_1_0.5,target_1_0.9
0,0.0,0.004199,NaN,NaN,NaN
1,1.0,0.004202,NaN,NaN,NaN
2,2.0,0.004204,NaN,NaN,NaN
3,3.0,0.004207,NaN,NaN,NaN
4,4.0,0.004209,NaN,NaN,NaN
...,...,...,...,...,...
257893,257893.0,0.411087,NaN,NaN,NaN
257894,257894.0,0.411091,NaN,NaN,NaN
257895,257895.0,0.411093,NaN,NaN,NaN
257896,257896.0,NaN,0.350142,0.359807,0.364757


In [42]:
# Generate some time series
N = 500
t = np.linspace(0, 1, N)
e = np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=N)
a = 10 + 10 * t + 10 * np.cos(2 * np.pi * (10 * t - 0.5)) + 1 * e[:, 0]
b = 20 + 20 * t + 20 * np.cos(2 * np.pi * (20 * t - 0.5)) + 2 * e[:, 1]
c = 30 + 30 * t + 30 * np.cos(2 * np.pi * (30 * t - 0.5)) + 3 * e[:, 2]
y = np.hstack([a.reshape(-1, 1), b.reshape(-1, 1), c.reshape(-1, 1)])

# Fit the model
model = DeepTCN(
    y=y,
    x=None,
    forecast_period=100,
    lookback_period=100,
    quantiles=[0.001, 0.1, 0.5, 0.9, 0.999],
    filters=3,
    kernel_size=3,
    dilation_rates=[1],
    loss='parametric'
)

model.fit(
    learning_rate=0.001,
    batch_size=16,
    epochs=300,
    verbose=1
)

# Generate the forecasts
df = model.forecast(y=y, x=None)
# Generate some time series
N = 500
t = np.linspace(0, 1, N)
e = np.random.multivariate_normal(mean=np.zeros(3), cov=np.eye(3), size=N)
a = 10 + 10 * t + 10 * np.cos(2 * np.pi * (10 * t - 0.5)) + 1 * e[:, 0]
b = 20 + 20 * t + 20 * np.cos(2 * np.pi * (20 * t - 0.5)) + 2 * e[:, 1]
c = 30 + 30 * t + 30 * np.cos(2 * np.pi * (30 * t - 0.5)) + 3 * e[:, 2]
y = np.hstack([a.reshape(-1, 1), b.reshape(-1, 1), c.reshape(-1, 1)])

# Fit the model
model = DeepTCN(
    y=y,
    x=None,
    forecast_period=100,
    lookback_period=100,
    quantiles=[0.001, 0.1, 0.5, 0.9, 0.999],
    filters=3,
    kernel_size=3,
    dilation_rates=[1],
    loss='parametric'
)

model.fit(
    learning_rate=0.001,
    batch_size=16,
    epochs=300,
    verbose=1
)

# Generate the forecasts
df = model.forecast(y=y, x=None)

print(y.shape)

# # Plot the forecasts
fig = plot_matplotlib(df=df, quantiles=[0.001, 0.1, 0.5, 0.9, 0.999], x_axis_label="time", y_axis_label="value")
fig.write_image('results.png', scale=4, height=900, width=700)


Epoch 1/300
19/19 [==============================] - 1s 2ms/step - loss: 4.5648
Epoch 2/300
19/19 [==============================] - 0s 2ms/step - loss: 3.8086
Epoch 3/300
19/19 [==============================] - 0s 2ms/step - loss: 3.5058
Epoch 4/300
19/19 [==============================] - 0s 2ms/step - loss: 3.3088
Epoch 5/300
19/19 [==============================] - 0s 2ms/step - loss: 3.1402
Epoch 6/300
19/19 [==============================] - 0s 2ms/step - loss: 2.9777
Epoch 7/300
19/19 [==============================] - 0s 2ms/step - loss: 2.8185
Epoch 8/300
19/19 [==============================] - 0s 3ms/step - loss: 2.6597
Epoch 9/300
19/19 [==============================] - 0s 2ms/step - loss: 2.5042
Epoch 10/300
19/19 [==============================] - 0s 2ms/step - loss: 2.3528
Epoch 11/300
19/19 [==============================] - 0s 3ms/step - loss: 2.2044
Epoch 12/300
19/19 [==============================] - 0s 2ms/step - loss: 2.0570
Epoch 13/300
19/19 [=================